In [1]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.utils.data

import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
file1= 'ptbdb_normal.csv'
file2= 'ptbdb_abnormal.csv'
file3= 'mitbih_test.csv'
file4= 'mitbih_train.csv'

torch.manual_seed(19)

dfnorm= pd.read_csv(file1, header=None,dtype=np.float32)
dfabnorm= pd.read_csv(file2, header=None,dtype=np.float32)
dftrain = pd.read_csv(file4, header=None,dtype=np.float32)
dftest = pd.read_csv(file3, header=None,dtype=np.float32) 

npnorm=dfnorm.values
npabnorm=dfabnorm.values



#TrainTor=torch.from_numpy(nptrain).double()
#TestTor=torch.from_numpy(nptest).double()
#valsize=int(TestTor.size()[0]*0.8)
#testsize=TestTor.size()[0]-valsize
#ValTor, TestTor=torch.utils.data.random_split(TestTor,(valsize,testsize) )



abnormTor=torch.from_numpy(npabnorm).double()
normTor= torch.from_numpy(npnorm).double()


A=torch.cat((normTor,abnormTor))
Train, ValTor=torch.utils.data.random_split(A,(9000,A.size()[0]-9000) )
ValTor,TestTor= torch.utils.data.random_split(ValTor,(int(0.8*(A.size()[0]-9000)),int(0.2*(A.size()[0]-9000)+1) ))

In [2]:
def outsize(insize,filtersize,stride,padding):
    x=float((insize-filtersize+2*padding))/stride +1
    return int(x)

bsize=int(4)
filter1=20
stride1=1
padding1=0
filter2=5
lin1=outsize(187,filter1,stride1,padding1)
lin2=25
lin3=25
out=2



In [3]:
trainloader = torch.utils.data.DataLoader(Train, batch_size=bsize,
                                          shuffle=True, num_workers=2,drop_last=True)

valloader = torch.utils.data.DataLoader(ValTor, batch_size=bsize,
                                         shuffle=False, num_workers=2,drop_last=True)

testloader = torch.utils.data.DataLoader(TestTor, batch_size=bsize,
                                         shuffle=False, num_workers=2,drop_last=True)

classes = ('0', '1') 
#%%


In [4]:
import torch.nn as nn
import torch.nn.functional as F

out1= outsize(187,filter1,stride1,padding1)
out2= outsize(out1,filter2,2,padding1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=5, kernel_size=filter1, stride=stride1).double()
        self.conv2 = nn.Conv1d(in_channels=5, out_channels=10,kernel_size=filter2,stride=2).double()
        self.fc1 = nn.Linear(out2*10, lin2).double()
        self.fc3 = nn.Linear(lin2, lin3).double()
        #self.fc1 = nn.Linear(out2*10, lin2).double()
        #self.fc2 = nn.Linear(lin2, lin3).double()
        self.fc3 = nn.Linear(lin3, out).double()

    def forward(self, x):
        x=x.view(bsize,1,187)
        x = F.relu(self.conv1(x))
        #x = F.relu(self.conv2(x))
        
        x=F.relu(self.conv2(x))
        
        x = x.view(-1,int(10*out2))
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
#%%
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.005)

#%%
for epoch in range(25):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs=data[:,:187]
        labels = data[:,187].long()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.594
[2,  2000] loss: 0.593
[3,  2000] loss: 0.591
[4,  2000] loss: 0.590
[5,  2000] loss: 0.595
[6,  2000] loss: 0.591
[7,  2000] loss: 0.592
[8,  2000] loss: 0.591
[9,  2000] loss: 0.593
[10,  2000] loss: 0.594
[11,  2000] loss: 0.530
[12,  2000] loss: 0.406
[13,  2000] loss: 0.374
[14,  2000] loss: 0.363
[15,  2000] loss: 0.343
[16,  2000] loss: 0.325
[17,  2000] loss: 0.312
[18,  2000] loss: 0.293
[19,  2000] loss: 0.284
[20,  2000] loss: 0.270
[21,  2000] loss: 0.256
[22,  2000] loss: 0.237
[23,  2000] loss: 0.245
[24,  2000] loss: 0.234
[25,  2000] loss: 0.231
Finished Training


In [7]:
#Validation:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    acc=0.0
    for i, data in enumerate(valloader, 0):
        # get the inputs
        inputs=data[:,:187]
        labels = data[:,187].long()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        acc+=float(sum(torch.max(net(inputs),1)[1]==labels))/labels.size()[0]
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            print('[%d, %5d] accuracy: %.3f' %
                  (epoch + 1, i + 1, acc / 100))
            acc=0.0
            running_loss = 0.0



[1,   100] loss: 0.050
[1,   100] accuracy: 0.887
[1,   200] loss: 0.056
[1,   200] accuracy: 0.890
[1,   300] loss: 0.072
[1,   300] accuracy: 0.850
[1,   400] loss: 0.054
[1,   400] accuracy: 0.887
[1,   500] loss: 0.052
[1,   500] accuracy: 0.892
[1,   600] loss: 0.050
[1,   600] accuracy: 0.902
[1,   700] loss: 0.052
[1,   700] accuracy: 0.897
[1,   800] loss: 0.048
[1,   800] accuracy: 0.907
[1,   900] loss: 0.043
[1,   900] accuracy: 0.925
[1,  1000] loss: 0.041
[1,  1000] accuracy: 0.922
[1,  1100] loss: 0.048
[1,  1100] accuracy: 0.910
